# Iterators

## Topics

- Stream larger-than-memory data through a pipeline
- Composable thanks to the iterator protocol

My favorite "feature" of pandas is that it's written in Python.
Python has great language-level features for handling streams of data
that may not fit in memory.
This can be a useful pre-processing step to reading the data into a DataFrame or
NumPy array.
You can get quite far using just the builtin data structures as David Beazley proves in [this PyData keynote](https://www.youtube.com/watch?v=lyDLAutA88s).

In [ ]:
import os
import gzip
from itertools import islice, takewhile

import numpy as np
import pandas as pd
import seaborn as sns
import dask.dataframe as dd
from toolz import partition_all, partitionby
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
pd.options.display.max_rows = 10
sns.set(context='talk')
plt.style.use("default")

## Beer Reviews Dataset

- A review is a list of lines
- Each review line is formated like `meta/field: value`
- Reviews are separated by blank lines (i.e. the line is just `'\n'`)


Stanford has a [dataset on beer reviews](https://snap.stanford.edu/data/web-BeerAdvocate.html). The raw file is too large for me to include, but I split off a couple subsets for us to work with.

Pandas can't read this file natively, but we have Python!
We'll use Python to parse the raw file and tranform it into a tabular format.

In [ ]:
with gzip.open("data/beer-raw-small.txt.gz", "r") as f:
    print(f.read(1500).decode('utf-8'))

The full compressed raw dataset is about 500MB, so reading it all into memory might not be pleasent (we're working with a small subset that would fit in memory, but pretend it didn't).
Fortunately, Python's iterator protocol and generators make dealing with large streams of data pleasent.

## Developing a solution

Let's build a solution together. I'll provide some guidance as we go along.

In [ ]:
# Get a handle to the data
f = gzip.open("data/beer-raw-small.txt.gz", "rt")

Usually you'd use a context manager like `with gzip.open(...) as f`, but for debugging, it's OK to do it this way.

## Parsing Tasks

1. split the raw text stream into individual reviews
2. transform each individual review into a data container
3. combine a chunk of transformed individual reviews into a collection
4. store the chunk to disk

Let's grab the first review using [`takewhile`](https://docs.python.org/3/library/itertools.html#itertools.takewhile) till the first `'\n'`.
`takewhile` scans a stream, returning each item (line) until it hits the sentinal value it's looking for.

In [ ]:
from itertools import takewhile

In [ ]:
f.seek(0);  # make the cell idempotent
first = list(takewhile(lambda x: x != '\n', f))
first

<div class="alert alert-success" data-title="Format Review">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Format Review</h1>
</div>
<p>Write a function `format_review` that converts an item like `first` into a dict</p>

It will have one entry per line, where the are the stuff to the left of the colon and the values are the stuff to the right.
For example, the first line would be

`'beer/name: Sausa Weizen\n',` => `'beer/name': 'Sausa Weizen'`

Make sure to clean up the line endings too.

- Hint: Check out the [python string methods](https://docs.python.org/3/library/stdtypes.html#string-methods)

You can check your function against `expected` by evaluating the next cell.
If you get a failure, adjust your `format_review` until it passes.

In [ ]:
import unittest
from typing import List, Dict

f.seek(0);  # make the cell idempotent
review = list(takewhile(lambda x: x != '\n', f))


def format_review(review: List[str]) -> Dict[str, str]:
    """Your code goes below"""
    

class TestFormat(unittest.TestCase):
    maxDiff = None

    def test_format_review(self):
        result = format_review(review)
        expected = {
            'beer/ABV': '5.00',
            'beer/beerId': '47986',
            'beer/brewerId': '10325',
            'beer/name': 'Sausa Weizen',
            'beer/style': 'Hefeweizen',
            'review/appearance': '2.5',
            'review/aroma': '2',
            'review/overall': '1.5',
            'review/palate': '1.5',
            'review/profileName': 'stcules',
            'review/taste': '1.5',
            'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.\t\t',
            'review/time': '1234817823'
        }
        self.assertEqual(result, expected)

suite = unittest.TestLoader().loadTestsFromModule(TestFormat())
unittest.TextTestRunner().run(suite)

In [ ]:
%load solutions/groupby_format_review.py

Notice that optional argument to split, which controls the number of splits made; If a review text had contained a literal `': '`, we'd be in trouble since it'd get split again.

Make sure you executed the above solution cell twice (first to load, second to execute) as we'll be using that `format_review` function down below

## To a DataFrame

Assuming we've processed many reviews into a list, we'll then build up a DataFrame.

In [ ]:
r = [format_review(first)]  # imagine a list of many reviews

col_names = {
    'beer/ABV': 'abv',
    'beer/beerId': 'beer_id',
    'beer/brewerId': 'brewer_id',
    'beer/name': 'beer_name',
    'beer/style': 'beer_style',
    'review/appearance': 'review_appearance',
    'review/aroma': 'review_aroma',
    'review/overall': 'review_overall',
    'review/palate': 'review_palate',
    'review/profileName': 'profile_name',
    'review/taste': 'review_taste',
    'review/text': 'text',
    'review/time': 'time'
}
df = pd.DataFrame(r)
numeric = ['abv', 'review_appearance', 'review_aroma',
           'review_overall', 'review_palate', 'review_taste']
df = (df.rename(columns=col_names)
        .replace('', np.nan))
df[numeric] = df[numeric].astype(float)
df['time'] = pd.to_datetime(df.time.astype(int), unit='s')
df

Again, writing that as a function:

In [ ]:
def as_dataframe(reviews):
    col_names = {
        'beer/ABV': 'abv',
        'beer/beerId': 'beer_id',
        'beer/brewerId': 'brewer_id',
        'beer/name': 'beer_name',
        'beer/style': 'beer_style',
        'review/appearance': 'review_appearance',
        'review/aroma': 'review_aroma',
        'review/overall': 'review_overall',
        'review/palate': 'review_palate',
        'review/profileName': 'profile_name',
        'review/taste': 'review_taste',
        'review/text': 'text',
        'review/time': 'time'
    }
    df = pd.DataFrame(list(reviews))
    numeric = ['abv', 'review_appearance', 'review_aroma',
               'review_overall', 'review_palate', 'review_taste']
    df = (df.rename(columns=col_names)
            .replace('', np.nan))
    df[numeric] = df[numeric].astype(float)
    df['time'] = pd.to_datetime(df.time.astype(int), unit='s')
    return df

## Full pipeline

1. `file -> review_lines : List[str]`
2. `review_lines -> reviews : Dict[str, str]`
3. `reviews -> DataFrames`
4. `DataFrames -> CSV`

The full pipeline would look something like:

In [ ]:
from toolz import partition_all, partitionby


BATCH_SIZE = 100  # Number of reviews to process per chunk
                  # Intentionally small for demostration    


with gzip.open("data/beer-raw-small.txt.gz", "rt") as f:

    # Filter out a null byte at the end
    lines = (x for x in f if not x.startswith('\x00'))
    
    review_lines_and_newlines = partitionby(lambda x: x == '\n', lines)
    # that goes [review, \n, review, \n, ...]
    # so filter out the newlines
    review_lines = filter(lambda x: x != ('\n',), review_lines_and_newlines)
    
    # generator expression to go from List[str] -> Dict[str, str]
    reviews = (format_review(x) for x in review_lines)
    
    # `reviews` yields one dict per review.
    # Won't fit in memory, so do `BATCH_SIZE` per chunk
    chunks = partition_all(BATCH_SIZE, reviews)
    dfs = (as_dataframe(chunk) for chunk in chunks)
    os.makedirs("data/beer/", exist_ok=True)

    # the first time we read from disk
    for i, df in enumerate(dfs):
        df.to_csv("data/beer/chunk_%s.csv.gz" % i, index=False,
                  compression="gzip")
        print(i, end='\r')


This runs comfortably in memory. At any given time, we only have `BATCH_SIZE` reviews in memory.

## Brief Aside on [Dask](http://dask.pydata.org/en/latest/)

> Dask is a flexible parallel computing library for analytic computing.

The original dataset is in random order, but I wanted to select an interesting subset for us to work on: all the reviews by the top 100 reviewers.

You might know enough pandas now to figure out the top-100 reviewers by count.
Do a `value_counts` , select the top 100, then select the index.

```python
top_reviewers = df.profile_name.value_counts().nlargest(100).index
```

Recall that `value_counts` will be a `Series` where the index is each unique `profile_name` and the values is the count of reviews for that profile.
We use `nlargets(100)` to get the 100 largest values, and `.index` to get the actual profile names. 

But that assumes we have a `df` containing the full dataset in memory.
My laptop can't load the entire dataset though (recall that we're working with a subset today).

It wouldn't be *that* hard to write a custom solution in python or pandas using chunking like we did up above.
We'd split our task into parts

- read a chunk
- compute `chunk.profile_name.value_counts()`
- store that intermediate `value_counts` in a global container

Once we've processed each chunk, our final steps are to

- merge each `value_counts` chunk by summing
- filter to the top 100

This pattern of processing chunks independently (map) and combining the results into a smaller output (reduce) is common, and doing it manually gets old.
Dask can help out here.

## Collections, Tasks, Schedulers

![](http://dask.pydata.org/en/latest/_images/collections-schedulers.png)

Dask has several components, so it can be hard to succinctly describe the library.
For our purposes, we'll view it as providing "big dataframes".
In reality, "big dataframes" means a collection of many smaller pandas `DataFrame`s, with algorithms for operating on them in parallel and combining the results.
You write python code that's quite similar to regular NumPy and pandas, only instead of execting immediatly a *task graph* (DAG for Directect Acylic Graph) is created.
Dask knows how to *schedule* the DAG to execute on some workers (the several cores or your laptop, or an entire distributed cluster).

In [ ]:
import dask.dataframe as dd

In [ ]:
a = dd.read_csv("data/beer/chunk*.csv.gz", compression="gzip", blocksize=None,
                parse_dates=['time'])
a.info()

That API should look familiar to you, now that you're experienced pandas users.
We swap out `pd` for `dd`, and stuff mostly just works.
Occasionally, you'll have a `dask`-specific thing like `blocksize` (number of bytes per smaller dataframe) that don't apply to pandas, which assumes things fit in memory.

In [ ]:
top_reviewers = a.profile_name.value_counts().nlargest(100).index
top_reviewers

We haven't really done any work yet. `dask` has just built the DAG representing the computation it will perform when needed.
We can directly visualize DAGs that aren't too large with the `.visualize` method.

We can visualize the DAG built up by dask with the `.visualize` method.
I've done this ahead of time, since then library that draws the graphs isn't part of the required packages.

```python
top_reviewers.visualize()
```
![](mydask.png)

In [ ]:
top_reviewers.compute()

We can use `top_reviewers` as a boolean mask, just like in regular pandas.

```python
>>> a[a.profile_name.isin(top_reviewers.compute())].to_csv("data/subset.csv.gz")
```

## Back to pandas

I've provided the reviews by the top 100 reviewers.
We'll use it for talking about groupby.

In [ ]:
df = pd.read_csv("data/subset.csv.gz", compression="gzip",
                 parse_dates=['time'])
df.head()

In [ ]:
df.info()

## Aside: Namespaces

Pandas has been expanding its use of namespaces (or accessors) on `DataFrame` to group together related methods. This also limits the number of methods direclty attached to `DataFrame` itself, which can be overwhelming.

Currently, we have these namespaces:

- `.str`: defined on `Series` and `Index`es containing strings (object dtype)
- `.dt`: defined on `Series` with `datetime` or `timedelta` dtype
- `.cat`: defined on `Series` and `Indexes` with `category` dtype
- `.plot`: defined on `Series` and `DataFrames`

<div class="alert alert-success" data-title="Reviews by Hour">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Reviews by Hour</h1>
</div>

<p>Make a barplot of the count of reviews by hour of the day.</p>

- Hint: Use the `.dt` namespace to get the `hour` component of a `datetime`
- Hint: We've seen `Series.value_counts` for getting the count of each value
- Hint: Use `.sort_index` to make sure the data is ordered by hour, not count
- Hint: Use the [`.plot`](http://pandas.pydata.org/pandas-docs/stable/api.html#plotting) namespace to get a `bar` chart

In [ ]:
%load solutions/groupby_03.py

<div class="alert alert-success" data-title="Pale Ales">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Pale Ales</h1>
</div>
<p>
Make a variable `pale_ales` that filters `df` to just rows where `beer_style` contains the string `'pale ale'` (ignoring case)
</p>
- Hint: Use the `df.beer_style.str` namespace and find a method for checking whether a string contains another string.

In [ ]:
%load solutions/groupby_04.py

# Groupby

Groupby operations come up in a lot of contexts.
At its root, groupby about doing an operation on many subsets of the data, each of which shares something in common.
The components of a groupby operation are:

## Components of a groupby

1. **split** a table into groups
2. **apply** a function to each group
3. **combine** the results into a single DataFrame or Series

In pandas the `split` step looks like

```python
df.groupby( grouper )
```

`grouper` can be many things

- Series (or string indicating a column in `df`)
- function (to be applied on the index)
- dict : groups by *values*
- `levels=[ names of levels in a MultiIndex ]`

## Split

Break a table into smaller logical tables according to some rule

In [ ]:
gr = df.groupby("beer_name")
gr

We haven't really done any actual work yet, but pandas knows what it needs to know to break the larger `df` into many smaller pieces, one for each distinct `beer_name`.

## Apply & Combine

To finish the groupby, we apply a method to the groupby object.

In [ ]:
review_cols = ['review_appearance', 'review_aroma', 'review_overall',
               'review_palate', 'review_taste']

df.groupby('beer_name')[review_cols].agg('mean')

In this case, the function we applied was `'mean'`.
Pandas has implemented cythonized versions of certain common methods like mean, sum, etc.
You can also pass in regular functions like `np.mean`.

In terms of split, apply, combine, split was `df.groupby('beer_name')`. 
We apply the `mean` function by passing in `'mean'`.
Finally, by using the `.agg` method (for aggregate) we tell pandas to combine the results with one output row per group.

You can also pass in regular functions like `np.mean`.

In [ ]:
df.groupby('beer_name')[review_cols].agg(np.mean).head()

Finally, [certain methods](http://pandas.pydata.org/pandas-docs/stable/api.html#id35) have been attached to `Groupby` objects.

In [ ]:
df.groupby('beer_name')[review_cols].mean()

<div class="alert alert-success" data-title="Highest Variance">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Highest Variance</h1>
</div>

<p>Find the `beer_style`s with the greatest variance in `abv`.</p>

- hint: `.std` calculates the standard deviation (`.var` for variance), and is available on `GroupBy` objects like `gr.abv`.
- hint: use `.sort_values` to sort a Series by the values (it took us a while to come up with that name)

In [ ]:
%load solutions/groupby_abv.py

## `.agg` output shape

The output shape is determined by the grouper, data, and aggregation

- Grouper: Controls the output index
    * single grouper -> Index
    * array-like grouper -> MultiIndex
- Subject (Groupee): Controls the output data values
    * single column -> Series (or DataFrame if multiple aggregations)
    * multiple columns -> DataFrame
- Aggregation: Controls the output columns
    * single aggfunc -> Index in the colums
    * multiple aggfuncs -> MultiIndex in the columns (Or 1-D Index if groupee is 1-D)


We'll go into MultiIndexes in a bit, but for know, think of them as regular Indexes with multiple levels (columns).

In [ ]:
# single grouper, single groupee, single aggregation
df.groupby('beer_style').review_overall.agg('mean')

In [ ]:
# multiple groupers, multiple groupee, single aggregation
df.groupby(['brewer_id', 'beer_name'])[review_cols].agg(['mean', 'min', 'max', 'std', 'count'])

<div class="alert alert-success" data-title="Rating by length">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Rating by length</h1>
</div>

<p>Plot the relationship between review length (number of characters) and average `reveiw_overall`.</p>

- Hint: use `.plot(style='k.')`
- We've grouped by columns so far, you can also group by any series with the same length

In [ ]:
%load solutions/groupby_00.py

<div class="alert alert-success" data-title="Reviews by Length">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Reviews by Length</h1>
</div>

<p>Find the relationship between review length (number of **words** and average `reveiw_overall`.)</p>

- Hint: You can pass a [regular expression](https://docs.python.org/3/howto/regex.html#matching-characters) to any of the `.str` methods.

In [ ]:
%load solutions/groupby_00b.py

<div class="alert alert-success" data-title="Rating by number of Reviews">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Rating by number of Reviews</h1>
</div>

<p>Find the relationship between the number of reviews for a beer and the average `review_overall`.</p>


In [ ]:
%load solutions/groupby_01.py

## Transform

A *transform* is a function whose output is the same shape as the input.

Recall that a groupby has three steps: split, apply, combine.
So far, all of the functions we've applied have been *aggregations*: the rule for "combine" is one row per group.

You can use `Groupby.transform` when you have an operation that should be done *groupwise*, but the result should be the same shape.
For example, suppose we wanted to normalize each reviewer's scores by their average score. 

In [ ]:
# Define demean(v: array) -> array
def demean(v):
    return v - v.mean()

Just calling `demean` on the entire Series will noramilze by the *global* average.

In [ ]:
demean(df.review_overall)

Now, let's demean each individual's reviews by their own average.
This could be useful if, for example, you were building a recommendation system.
A rating of 4 from someone's whose average is 2 is in some sense more meaningful that a 4 from someone who always gives 4s.

In [ ]:
normalized = df.groupby("profile_name")[review_cols].transform(demean)
normalized.head()

We used `.transform` because the desired output was the same shape as the input.
Just like `.agg` informs pandas that you want `1 input group → 1 output row`, the `.transform` method informs pandas that you want `1 input row → 1 output row`.

`.transform` operates on each column independently.

<div class="alert alert-success" data-title="Personal Trend?">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Personal Trend?</h1>
</div>

<p>Do reviewer's `review_overall` trend over a person's time reviewing?</p>

Hint: Need an indictor that tracks which review this is for that person. That is, we need a cumulative count of reviews per person.

In [ ]:
%load solutions/groupby_02.py

## General `.apply`

We've seen `.agg` for outputting 1 row per group, and `.transform` for outputting 1 row per input row.

The final kind of function application is `.apply`.
This can do pretty much whatever you want.
We'll see an example in a later notebook.

## Summary

- We used Python's iterator protocol to transform the raw data to a table
- We saw how Dask could handle larger-than-memory data with a familiar API
- We used groupby to analyze data by subsets